# Minimax Depth vs Move-Time Analysis
In this notebook, we explore the relationship between the **depth** parameter of the Minimax algorithm and the corresponding **move-time** during gameplay in Othello. Essentially, we must understand the time it takes for our AI to make moves. The core hypothesis is based on the theoretical understanding that the complexity of the Minimax algorithm, particularly in terms of search time, grows *exponentially* with depth, i.e. $\mathcal{O}(b^d)$, where $b$ is the breadth of the game tree and $d$ is the depth. Here, "move-time" essentially refers to the time taken for a complete search cycle at a given depth. Through systematic data collection and analysis from Othello matches using Minimax agents, we aim to empirically validate the theoretically anticipated exponential growth pattern of move-time with increasing depth.

## Theoretical Analysis
The theoretical analysis aims to develop an understanding of how the Minimax algorithm's move-time (incorporating the search time) scales with varying depths in the game of Othello, focusing on the hypothesized relationship $f(d) = T \cdot b^d$, where $f(d)$ denotes the move-time at depth $d$.


- **Hypothesized Relationship:** Let $f(d)$ be the move-time for the Minimax algorithm at depth $d$. The Minimax algorithm's time complexity as a function of depth is expected to follow: $f(d) = T \cdot b^d$, where $T$ represents the average time taken to evaluate a single node in the search tree, and $b$ is the branching factor (approximately $b=10$ for Othello [1]). The key aspect of this relationship is its expected exponential nature.

- **Parameter Estimation:** To explore this hypothesized relationship, we plan to analyse $f(d)$ at depth 1 to establish a baseline: $f(1) = T \cdot b$. With the hypothesized relationship and reported branching factor from the literature ($b = 10$), we aim to estimate $T$.

- **Validation:** Post estimation, empirical validation will be conducted to compare our theoretical assumptions with observed data. Successful validation will confirm our understanding of how Minimax's move-time scales with depth in Othello.

**References:**

[1] Norvig, P. (1992) '[Search and the Game of Othello](https://www.sciencedirect.com/science/article/abs/pii/B9780080571157500182)', in Paradigms of Artificial Intelligence Programming: Case Studies in Common Lisp, pp. 596-654. doi: 10.1016/B978-0-08-057115-7.50018-2.
